In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
music_df = pd.read_csv('music_clean.csv')
music_df.head()

,Unnamed: 0,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,36506,60.0,0.896000,0.726,214547.0,0.177,0.000002,0.1160,-14.824,0.0353,92.934,0.618,1
1,37591,63.0,0.003840,0.635,190448.0,0.908,0.083400,0.2390,-4.795,0.0563,110.012,0.637,1
2,37658,59.0,0.000075,0.352,456320.0,0.956,0.020300,0.1250,-3.634,0.1490,122.897,0.228,1
3,36060,54.0,0.945000,0.488,352280.0,0.326,0.015700,0.1190,-12.020,0.0328,106.063,0.323,1
4,35710,55.0,0.245000,0.667,273693.0,0.647,0.000297,0.0633,-7.787,0.0487,143.995,0.300,1


In [3]:
print(music_df.isna().sum().sort_values())

Unnamed: 0          0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
genre               0
dtype: int64


In [4]:
#remove observations that account for less than 5% of all missing data
#subset is list of columns that all have less than 5% missing values
#if any observation is to found to have a blank value in any of these columns, the whole row is dropped

In [5]:
music_df = music_df.dropna(subset=[
    'genre',
    'popularity',
    'loudness',
    'liveness',
    'tempo'
])
print(music_df.isna().sum().sort_values())

Unnamed: 0          0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
genre               0
dtype: int64


In [6]:
#can also impute missing values, meaning make an educated guess based on mean, median or such
#always impute after splitting data for testing and training

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
X_cat = music_df['genre'].values.reshape(-1, 1)
X_num = music_df.drop(['genre','popularity'], axis=1).values
y = music_df['popularity'].values

In [11]:
from sklearn.model_selection import train_test_split

X_train_cat, X_test_cat, y_train, y_test = train_test_split(X_cat, y, test_size=0.2, random_state=12)
X_train_num, X_test_num, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=12)

imp_cat = SimpleImputer(strategy='most_frequent')
X_train_cat = imp_cat.fit_transform(X_train_cat)
X_test_cat = imp_cat.transform(X_test_cat)

In [12]:
imp_num = SimpleImputer()

In [13]:
X_train_num = imp_num.fit_transform(X_train_num)
X_test_num = imp_num.transform(X_test_num)
X_train = np.append(X_train_num, X_train_cat, axis=1)
X_test = np.append(X_test_num, X_test_cat, axis=1)

In [14]:
#alternatively, impute using pipeline, which makes everything faster

In [18]:
from sklearn.pipeline import Pipeline

music_df = music_df.dropna(subset=[
    'genre',
    'popularity',
    'loudness',
    'liveness',
    'tempo'
])
music_df['genre'] = np.where(music_df['genre'] == 'Rock', 1, 0)
X = music_df.drop('genre', axis=1).values
y = music_df['genre'].values
print(X.shape, y.shape)

(1000, 12) (1000,)


In [19]:
from sklearn.linear_model import LogisticRegression

steps = [('imputation', SimpleImputer()),
        ('logistic_regression', LogisticRegression())]
pipeline = Pipeline(steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

Pipeline(steps=[('imputation', SimpleImputer()),
                ('logistic_regression', LogisticRegression())])


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0